# Preenchimento híbrido das tabelas de fluxo
Considerando os dois tipos de preenchimento de tabelas, proativo e reativo, podemos utilizar uma abordagem híbrida, onde o controlador inicia a rede preenchendo todas as tabelas dos nós baseando-se em informações pré-definidas. Em seguida, a rede utiliza do preenchimento reativo, onde o controlador define as novas linhas das tabelas se baseando nos requests e informações da rede.

In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request

In [2]:
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
#rede.draw()

Hosts inicializados
Canais inicializados
Pares EPRs adicionados


### Preenchendo com modo proativo

In [3]:
# Preenchimento proativo da tabela de fluxos
hosts = rede.hosts
for alice in hosts:
    print("Preenchendo a tabela do Host", alice)
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice=alice, bob=bob, fmin=0.5, neprs=2)
        controlador.add_match_route_rule_in_host_proactive(alice=alice, bob=bob, fmin=0.9, neprs=3)

Preenchendo a tabela do Host 0
Preenchendo a tabela do Host 1
Preenchendo a tabela do Host 2
Preenchendo a tabela do Host 3
Preenchendo a tabela do Host 4
Preenchendo a tabela do Host 5
Preenchendo a tabela do Host 6
Preenchendo a tabela do Host 7
Preenchendo a tabela do Host 8
Preenchendo a tabela do Host 9
Preenchendo a tabela do Host 10
Preenchendo a tabela do Host 11


In [4]:
# Tabelas de fluxo após preenchimento
for host in hosts:
    print(f"Tabela de fluxo do Host {host}:")
    rede.get_host(host).draw_flow_table()

Tabela de fluxo do Host 0:
+--------------+----------------------+--------------------+
| Match        | Route                | Rule               |
+==============+======================+====================+
| (0, 0.5, 2)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (0, 0.9, 3)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.5, 2)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.9, 3)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.5, 2)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.9, 3)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (3, 0.5, 2)  | [0, 1, 2, 3]         | BasicRuleProactive

### Preenchendo com modo reativo

In [5]:
# Criação de requisições
num_hosts = len(rede.hosts) - 1
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
print("Hosts:", num_hosts)
requests = list(generate_random_request(num_hosts, fmin_range, neprs_range) for i in range(num_requests))
print("Requisições:", requests)

Hosts: 11
Requisições: [<quantumnet.objects.request.Request object at 0x00000218A89C81A0>, <quantumnet.objects.request.Request object at 0x00000218A84BA990>, <quantumnet.objects.request.Request object at 0x00000218A84BAAD0>, <quantumnet.objects.request.Request object at 0x00000218A84AF490>, <quantumnet.objects.request.Request object at 0x00000218A84AF5C0>, <quantumnet.objects.request.Request object at 0x00000218A7F12C30>, <quantumnet.objects.request.Request object at 0x00000218A8513350>, <quantumnet.objects.request.Request object at 0x00000218A8513460>, <quantumnet.objects.request.Request object at 0x00000218A854D650>, <quantumnet.objects.request.Request object at 0x00000218A854D850>, <quantumnet.objects.request.Request object at 0x00000218A858C5F0>, <quantumnet.objects.request.Request object at 0x00000218A858CF50>, <quantumnet.objects.request.Request object at 0x00000218A89D0F30>, <quantumnet.objects.request.Request object at 0x00000218A89D11D0>, <quantumnet.objects.request.Request ob

In [6]:
# Percorrer todas as requests
for request in requests:
    alice = rede.get_host(request.alice)
    # Encontra a regra na tabela de fluxo, caso não exista, adiciona e executa
    rule = alice.find_rule_by_request(request)
    if rule == False:
        controlador.add_match_route_rule_in_host_reactive(request)
    else:
        controlador.run_rule(rule[1])

Decisão aplicada: (High Fidelity,)
Decisão aplicada: (Normal E2E,)
Decisão aplicada: (Normal E2E,)
Decisão aplicada: (High Fidelity,)
Decisão aplicada: (High Fidelity,)
Decisão aplicada: (High Fidelity,)
Decisão aplicada: (Normal E2E,)
Tempo: 1
Executando ação: CreateEPR(1-5)
Executando ação: CreateEPR(5-6)
Executando ação: CreateEPR(6-7)
Tempo: 2
Executando ação: Swap(1-5-6)
Running Swap(1-5-6)
Tempo: 1
Executando ação: CreateEPR(5-4)
Decisão aplicada: (Normal E2E,)
Tempo: 1
Executando ação: CreateEPR(5-4)
Decisão aplicada: (Normal E2E,)
Tempo: 1
Executando ação: CreateEPR(8-4)
Executando ação: CreateEPR(4-5)
Tempo: 2
Executando ação: Swap(8-4-5)
Running Swap(8-4-5)
Decisão aplicada: (Normal E2E,)
Tempo: 1
Executando ação: CreateEPR(6-5)
Executando ação: CreateEPR(5-4)
Tempo: 2
Executando ação: Swap(6-5-4)
Running Swap(6-5-4)
Tempo: 1
Executando ação: CreateEPR(10-9)
Executando ação: CreateEPR(9-8)
Tempo: 2
Executando ação: Swap(10-9-8)
Running Swap(10-9-8)
Tempo: 1
Executando ação: C

In [7]:
# Tabelas de fluxo após preenchimento
for host in hosts:
    print(f"Tabela de fluxo do Host {host}:")
    rede.get_host(host).draw_flow_table()

Tabela de fluxo do Host 0:
+---------------+----------------------+--------------------+
| Match         | Route                | Rule               |
+===============+======================+====================+
| (0, 0.5, 2)   | [0]                  | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (0, 0.9, 3)   | [0]                  | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (1, 0.5, 2)   | [0, 1]               | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (1, 0.9, 3)   | [0, 1]               | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (2, 0.5, 2)   | [0, 1, 2]            | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (2, 0.9, 3)   | [0, 1, 2]            | BasicRuleProactive |
+---------------+----------------------+--------------------+
| (3, 0.5, 2)   | [0, 1, 2, 3]         | Ba